# 2-4. RAG - Embedding
https://wikidocs.net/231431


In [5]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [2]:
!pip install -q langchain_google_genai


In [8]:
# 구글 인증키 로딩
import os, dotenv
dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# GEMINI_API_KEY # 이 출력 결과도 커밋이 되지 않도록 해야 함.

# langchain 객체들은 환경변수에 저장된 api key를 제대로 인식할 수 있다.
# 이렇게 해 두면 매번 모델 생성 시 api key를 전달하지 않아도 되니 편하다.
#
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
None

2-4-2.

## HuggingFaceEmbeddings

In [2]:
!pip install -q sentence-transformers


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli', # 한국어 자연어 추론에 최적화 된 모델이라고 함.
    model_kwargs={'device':'cpu'},  # 나는 cuda 가 아니니...
    encode_kwargs={'normalize_embeddings':True},
)
# 주의: 이 셀 실행에 시간이 많이 걸린다. 약 45초.
embeddings_model

/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_18589/251242760.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
/Users/yhlee/miniforge3/envs/ml_langchain/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='jhgan/ko-sroberta-nli', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [4]:
embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)
len(embeddings), len(embeddings[0])

(5, 768)

In [ ]:
embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

0.5899015798177862
0.41826302433660056
0.7240604100932585
0.057026555734256024
0.4316417109086967


2-4-3
## GoogleGenerativeAIEmbeddings


In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

# 핵심 클래스는 GoogleGenerativeAIEmbeddings 이다.
# 그런데 지정하는 '모델' 이름이 'embedding..' 이다? gemini 같은 것이 아니다?

print(type(embeddings_model))
embeddings_model

<class 'langchain_google_genai.embeddings.GoogleGenerativeAIEmbeddings'>


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x116157e50>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [ ]:
embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)
print(type(embeddings), type(embeddings[0])) # 타입은?

print(len(embeddings))  # 임베딩 된 문서의 총 수. 위에서 제공한 리스트의 요소의 수?

print(len(embeddings[0])) # 이게 왜 차원을 나타낸다고 하는 것인가?
# 768

print([ len(e) for e in embeddings ])
# [768, 768, 768, 768, 768]
# 모든 임베딩이 동일한 차원(?) 을 가지고 있다.

<class 'list'> <class 'list'>
5
768
[768, 768, 768, 768, 768]


In [19]:
# embeddings[0]
'''
다음과 같이 표시되는 다차원 벡터. 튜플이 아니라 리스트 형식이다.

[0.03633681312203407,
 0.006253337021917105,
 -0.06295967847108841,
...
 0.0001472131407354027,
 0.02952706813812256]
'''
None

In [18]:
# 주어진 쿼리에 대한 임베딩을 생성.
# embed_documents() 와 embed_query() 를 구분하는 이유는??

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

'''
0.6051637037092901   '안녕하세요!',
0.6202318357552694   '어! 오랜만이에요',
0.7160855818278375   '이름이 어떻게 되세요?',  <= 제일 큰 값
0.6574387493658048   '날씨가 추워요',
0.555273899712489    'Hello LLM!'
'''
None

0.6051637037092901
0.6202318357552694
0.7160855818278375
0.6574387493658048
0.555273899712489
